The final part of week 3 on which i working with clusters of Toronto

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Importing the last data set with all the features from the previous steps

In [6]:

df=pd.read_csv("C:/Users/Spyros_PC/Desktop/IBM CERT/last.csv")
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Choosing to work only with Boroughs containing the word Toronto.

In [7]:
df_tor=df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Creating a map which includes marks for every Brrough location containing the word Toronto.
Each marker has a popup text which contains all the Neighbourhoods and the Borough at the end.

In [8]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

Storing the Forsquare Credentials

In [9]:
CLIENT_ID = 'ET0WGFUJUCKRKKNBG4AUPDUMV24FNN3A53UNITL3YPHNV4ZQ' # your Foursquare ID
CLIENT_SECRET = 'GRLHUEH2OG2T5I5MNPYFL0B5RP4ZD3I3T3BFTBDGLOA35KVC' # your Foursquare Secret
ACCESS_TOKEN = 'XBLN52XN45AX2BMUPBWRNN0DTHJCHIJVXCDQDDKEVXMKQVGM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ET0WGFUJUCKRKKNBG4AUPDUMV24FNN3A53UNITL3YPHNV4ZQ
CLIENT_SECRET:GRLHUEH2OG2T5I5MNPYFL0B5RP4ZD3I3T3BFTBDGLOA35KVC


In [10]:
address = '100 Front St W, Toronto, ON M5J 1E3, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.64590915 -79.3813637083646


1. Search for a specific venue category

Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel

In [11]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

results = requests.get(url).json()
results

Italian .... OK!


{'meta': {'code': 200, 'requestId': '601d5a6cd8e9d44e78321d1a'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5b897e92db1d81002c91df8c',
    'name': 'Fabbrica Rustic Italian',
    'location': {'address': '66 Wellington St W',
     'lat': 43.647161,
     'lng': -79.381691,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.647161,
       'lng': -79.381691}],
     'distance': 141,
     'postalCode': 'M5K 1E7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['66 Wellington St W',
      'Toronto ON M5K 1E7',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1612536428',
 

Get relevant part of JSON and transform it into a pandas dataframe

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,5b897e92db1d81002c91df8c,Fabbrica Rustic Italian,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612536428,False,66 Wellington St W,43.647161,-79.381691,"[{'label': 'display', 'lat': 43.647161, 'lng':...",141,M5K 1E7,CA,Toronto,ON,Canada,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]",NaN,NaN
1,581cad6a7c74e15859a6f890,Scaddabush Italian Kitchen & Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612536428,False,"200 Front Street W., Unit #G001",43.644737,-79.385355,"[{'label': 'display', 'lat': 43.6447367776608,...",346,M5V 3J1,CA,Toronto,ON,Canada,"[200 Front Street W., Unit #G001 (at Simcoe St...",at Simcoe St,NaN
2,4b7d777df964a52043c02fe3,Sauté Rosé,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612536428,False,55 University Ave.,43.646547,-79.384029,"[{'label': 'display', 'lat': 43.64654731418263...",226,M5J 2H7,CA,Toronto,ON,Canada,"[55 University Ave. (in Markel Building), Toro...",in Markel Building,NaN
3,4b846dd4f964a520dc3431e3,Little Anthony's,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612536428,False,121 Richmond St. W,43.650296,-79.384513,"[{'label': 'display', 'lat': 43.65029624519052...",550,M5H 2K1,CA,Toronto,ON,Canada,"[121 Richmond St. W (at York St.), Toronto ON ...",at York St.,75877692


In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Fabbrica Rustic Italian,Italian Restaurant,66 Wellington St W,43.647161,-79.381691,"[{'label': 'display', 'lat': 43.647161, 'lng':...",141,M5K 1E7,CA,Toronto,ON,Canada,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]",NaN,5b897e92db1d81002c91df8c
1,Scaddabush Italian Kitchen & Bar,Italian Restaurant,"200 Front Street W., Unit #G001",43.644737,-79.385355,"[{'label': 'display', 'lat': 43.6447367776608,...",346,M5V 3J1,CA,Toronto,ON,Canada,"[200 Front Street W., Unit #G001 (at Simcoe St...",at Simcoe St,581cad6a7c74e15859a6f890
2,Sauté Rosé,Italian Restaurant,55 University Ave.,43.646547,-79.384029,"[{'label': 'display', 'lat': 43.64654731418263...",226,M5J 2H7,CA,Toronto,ON,Canada,"[55 University Ave. (in Markel Building), Toro...",in Markel Building,4b7d777df964a52043c02fe3
3,Little Anthony's,Italian Restaurant,121 Richmond St. W,43.650296,-79.384513,"[{'label': 'display', 'lat': 43.65029624519052...",550,M5H 2K1,CA,Toronto,ON,Canada,"[121 Richmond St. W (at York St.), Toronto ON ...",at York St.,4b846dd4f964a520dc3431e3


 visualize the Italian restaurants that are nearby

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Fairmont Royal York Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

2. Explore a Given Venue

A. Let's explore the closests restaurants --Fabbrica Rustic Italian, Scaddabush Italian Kitchen & Bar, Sauté Rosé, Little Anthony's
We will get the ratings of the closest Italian restaurants so we can choose the best one

In [15]:
venue_id = ['5b897e92db1d81002c91df8c','581cad6a7c74e15859a6f890','4b7d777df964a52043c02fe3','4b846dd4f964a520dc3431e3'] # ID of Harry's Italian Pizza Bar

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id[0], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()
    
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'parent', 'hierarchy', 'attributes'])


{'id': '5b897e92db1d81002c91df8c',
 'name': 'Fabbrica Rustic Italian',
 'contact': {},
 'location': {'address': '66 Wellington St W',
  'lat': 43.647161,
  'lng': -79.381691,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.647161,
    'lng': -79.381691}],
  'postalCode': 'M5K 1E7',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['66 Wellington St W', 'Toronto ON M5K 1E7', 'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/fabbrica-rustic-italian/5b897e92db1d81002c91df8c',
 'categories': [{'id': '4bf58dd8d48988d110941735',
   'name': 'Italian Restaurant',
   'pluralName': 'Italian Restaurants',
   'shortName': 'Italian',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/

B. Get the venue's overall rating

In [16]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [17]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id[1], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()
    
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


In [18]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id[2], CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
result = requests.get(url).json()
    
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


So we can conclude that the highest rated reataurant is Scaddabush Italian Kitchen & Bar with rating 7.2